In [2]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [ ]:
#Asignamos valores a los parametros"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

# Torreon

In [45]:
metas = pd.DataFrame([('Avon',990000),('Oriflame',95000),('Invex_RLN',270000),('Invex_RLA',479000),('Invex_REL',470000),('Arabela',95000),('Cristian Lay',6000)])
metas.columns = ['Cliente','Meta']

In [4]:
#Lectura pagos Oriflam
pagos_oriflame = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Oriflame/RECUPERACION EXITOSA 18-12-20.xlsx')
pagos_oriflame['Cliente'] = 'Oriflame'
pagos_oriflame = pagos_oriflame.drop_duplicates()
pagos_oriflame = pagos_oriflame.loc[pagos_oriflame['FECHA_PAGO']>=mes3]

In [29]:
#Lectura pagos Arabela
pagos_arabela = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Arabela/18122020_PAGOS_ARABELA.xlsx')
pagos_arabela['Cliente'] = 'Arabela'
pagos_arabela = pagos_arabela.drop_duplicates()
pagos_arabela['FechaEntrega'] = [datetime.strptime(str(x), '%Y%m%d').strftime('%d/%m/%Y') for x in pagos_arabela['FechaEntrega']]
pagos_arabela['FechaEntrega'] = pd.to_datetime(pagos_arabela['FechaEntrega'],dayfirst=True)
pagos_arabela = pagos_arabela.loc[pagos_arabela['FechaEntrega']>=mes3]

In [17]:
#Lectura pagos invex
pagos_invex = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Invex/Pagos Invex.xlsx')
pagos_invex['Cliente'] = 'Invex_'+pagos_invex['Agencia'].map(str)
pagos_invex = pagos_invex.drop_duplicates()
pagos_invex = pagos_invex.loc[pagos_invex['TRAN_POST_DT']>=mes3]

# Oriflame

In [74]:
reporte_pagos_oriflame = pd.pivot_table(pagos_oriflame,index=['Cliente'],values=['fact'],aggfunc=np.sum)
reporte_pagos_oriflame = pd.DataFrame(reporte_pagos_oriflame.to_records())
reporte_pagos_oriflame.columns = ['Cliente','Facturacion']
reporte_pagos_oriflame['FechaFact'] = max(pagos_oriflame['FECHA_PAGO'])
reporte_pagos_oriflame['FechaFact'] = pd.to_datetime(reporte_pagos_oriflame['FechaFact'],dayfirst=True)
reporte_pagos_oriflame

,Cliente,Facturacion,FechaFact
0,Oriflame,39431.084,2020-12-18


# Invex

In [75]:
reporte_pagos_invex = pd.pivot_table(pagos_invex,index=['Cliente'],values=['AMT_TRAN','HONORARIO'],aggfunc=[np.sum,np.mean])
reporte_pagos_invex = pd.DataFrame(reporte_pagos_invex.to_records())
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,1,4]]
reporte_pagos_invex.columns = ['Cliente','Pago','Honorario']
reporte_pagos_invex['Facturacion'] = reporte_pagos_invex['Pago']*reporte_pagos_invex['Honorario']
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,3]]
reporte_pagos_invex['FechaFact'] = max(pagos_invex['TRAN_POST_DT'])
reporte_pagos_invex['FechaFact'] = pd.to_datetime(reporte_pagos_invex['FechaFact'],dayfirst=True)
reporte_pagos_invex

,Cliente,Facturacion,FechaFact
0,Invex_REL,308241.376,2020-12-18
1,Invex_RLA,354848.104,2020-12-18
2,Invex_RLN,140678.700,2020-12-18


# Arabela

In [73]:
reporte_pagos_arabela = pd.pivot_table(pagos_arabela,index=['Cliente'],values=['factura'],aggfunc=np.sum)
reporte_pagos_arabela = pd.DataFrame(reporte_pagos_arabela.to_records())
reporte_pagos_arabela.columns = ['Cliente','Facturacion']
reporte_pagos_arabela['FechaFact'] = max(pagos_arabela['FechaEntrega'])
reporte_pagos_arabela['FechaFact'] = pd.to_datetime(reporte_pagos_arabela['FechaFact'],dayfirst=True)
reporte_pagos_arabela

,Cliente,Facturacion,FechaFact
0,Arabela,53221.4916,2020-12-18


# Union sucursal

In [76]:
pdList = [reporte_pagos_arabela,reporte_pagos_invex,reporte_pagos_oriflame]
final_torreon = pd.concat(pdList,ignore_index=True)
final_torreon = pd.merge(final_torreon,metas,on=['Cliente'],how='left')
final_torreon['Avance%'] = final_torreon['Facturacion']/final_torreon['Meta']
final_torreon.insert(loc=0,column='Sucursal',value='Torreon')
final_torreon

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%
0,Torreon,Arabela,53221.4916,2020-12-18,95000,0.560226
1,Torreon,Invex_REL,308241.3760,2020-12-18,470000,0.655833
2,Torreon,Invex_RLA,354848.1040,2020-12-18,479000,0.740810
3,Torreon,Invex_RLN,140678.7000,2020-12-18,270000,0.521032
4,Torreon,Oriflame,39431.0840,2020-12-18,95000,0.415064


# Naucalpan

In [51]:
#Lectura pagos Bradesco
pagos_brad = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Bradesco/Pagos/ENRIQUEZ Acumulado de pagos Diciembre de 2020.xlsx')
pagos_brad = pagos_brad.iloc[:,0:9]
pagos_brad = pagos_brad.drop_duplicates()
pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [64]:
#Lectura pagos Liverpool
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
for i in sheet_liv:
    pagos_liv_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Liverpool/Pagos/PAGOS DIC LIVERPOOL .xlsx',sheet_name = str(i))
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
pagos_liv = pagos_liv.drop_duplicates()
pagos_liv = pagos_liv.iloc[:,0:6]
pagos_liv.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
try: 
    pagos_liv['Fecha Prom'] = pd.to_datetime(pagos_liv['Fecha Prom'],dayfirst=True)
except: pagos_liv['Fecha Prom'] = pagos_liv['Fecha Prom']
pagos_liv['Fecha Prom'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Prom']]
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
pagos_liv2 = pagos_liv2.loc[pagos_liv2['Fecha Pago']>=mes3]

In [53]:
#Lectura pagos Credifiel
sheet_cred = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for i in sheet_cred:
    pagos_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/'+i+'')
    pagos_cred = pd.concat([pagos_cred,pagos_cred_new],ignore_index=True)
pagos_cred = pagos_cred.iloc[:,0:5]
pagos_cred2 = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred2 = pd.DataFrame(pagos_cred2.to_records())

In [54]:
#Lectura asignacion Credifiel
sheet_cred2 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Asignacion/')
asignacion_cred = pd.DataFrame()
for j in sheet_cred2:
    asignacion_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Asignacion/'+j+'')
    asignacion_cred = pd.concat([asignacion_cred,asignacion_cred_new],ignore_index=True)
asignacion_cred = asignacion_cred.loc[:,['CREDITO','ARANCEL']]

In [55]:
#Lectura pagos Banco Azteca
sheet_baz_renta = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta')
pagos_baz_renta = pd.DataFrame()
for i in sheet_baz_renta:
    pagos_baz_renta_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta/'+i+'')
    pagos_baz_renta = pd.concat([pagos_baz_renta,pagos_baz_renta_new],ignore_index=True)
pagos_baz_renta = pagos_baz_renta.drop_duplicates()
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FNMONTOPAGADO']>0]
pagos_baz_renta['FDFECHAABONO'] = pd.to_datetime(pagos_baz_renta['FDFECHAABONO'],dayfirst=True)
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FDFECHAABONO']>=mes3]
pagos_baz_renta2 = pd.pivot_table(pagos_baz_renta,index=['CAMPANAID','FDFECHAABONO','FIIDPERIODO','FNSCOMPROMISO','FCEMPNUMCORTE','FNMONTOPROMETIDO'],values=['FNMONTOPAGADO'],aggfunc=np.sum)
pagos_baz_renta2 = pd.DataFrame(pagos_baz_renta2.to_records())
pagos_baz_renta2['Cliente'] = 'Renta Azteca'

In [56]:
#Leemos facturacion Banco Azteca
sheet_baz2 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/')
fact_baz = pd.DataFrame()
for k in sheet_baz2:
    fact_baz_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/'+k+'')
    fact_baz = pd.concat([fact_baz,fact_baz_new],ignore_index=True)
fact_baz = fact_baz.drop_duplicates()
fact_baz = fact_baz.loc[fact_baz['Número Semana']>=prim_sem_mes]
fact_baz['Final de Semana'] = pd.to_datetime(fact_baz['Final de Semana'],dayfirst=True)
fact_baz['Inicio de Semana'] = pd.to_datetime(fact_baz['Inicio de Semana'],dayfirst=True)
fact_baz['Cliente'] = 'Banco Azteca'

In [57]:
#Lectura pagos Banco Azteca Recovery
sheet_baz_recovery = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/')
pagos_baz_recovery = pd.DataFrame()
for k in sheet_baz_recovery:
    pagos_baz_recovery_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/'+k+'',sep='|')
    pagos_baz_recovery = pd.concat([pagos_baz_recovery,pagos_baz_recovery_new],ignore_index=True)
pagos_baz_recovery['Cartera'] = 'Recovery'
pagos_baz_recovery['DIA'] = pd.to_datetime(pagos_baz_recovery['DIA'],dayfirst=True)
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery['credito'] = pagos_baz_recovery['PAIS'].map(str)+'-'+pagos_baz_recovery['CANAL'].map(str)+'-'+pagos_baz_recovery['SUCURSAL'].map(str)+'-'+pagos_baz_recovery['FOLIO'].map(str)
pagos_baz_recovery = pagos_baz_recovery.drop_duplicates()

In [58]:
#Lectura pagos Banco Azteca 26-39
sheet_baz_26 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/')
pagos_baz_26 = pd.DataFrame()
for j in sheet_baz_26:
    pagos_baz_26_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/'+j+'',sep='|')
    pagos_baz_26 = pd.concat([pagos_baz_26,pagos_baz_26_new],ignore_index=True)
pagos_baz_26['cartera'] = '26-39'
pagos_baz_26['DIA'] = pd.to_datetime(pagos_baz_26['DIA'],dayfirst=True)
pagos_baz_26 = pagos_baz_26.loc[pagos_baz_26['DIA']>=mes3]
pagos_baz_26['credito'] = pagos_baz_26['PAIS'].map(str)+'-'+pagos_baz_26['CANAL'].map(str)+'-'+pagos_baz_26['SUCURSAL'].map(str)+'-'+pagos_baz_26['FOLIO'].map(str)
pagos_baz_26 = pagos_baz_26.drop_duplicates()

# Liverpool

In [59]:
meta_naucalpan = pd.DataFrame([('Liverpool',1000000),('Bradesco',1200000),('Credifiel',120000)])
meta_naucalpan.columns = ['Cliente','Meta']

In [65]:
#Asignamos los aranceles de liverpool
aranceles = [('DKN00REM',0.28),('DKN02REM',0.4),('DKN04REM',0.5),('SA001REM',0.15),('SA002REM',0.16)]
aranceles = pd.DataFrame(aranceles)
aranceles.columns = ['Id','Arancel']

#Agregamos aranceles al consolidado
pagos_liv2 = pd.merge(pagos_liv2,aranceles,how='left',on='Id')
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]
pagos_liv2['Fecha Pago'] = pd.to_datetime(pagos_liv2['Fecha Pago'],dayfirst=True)
pagos_liv2['Cliente'] = 'Liverpool'

In [71]:
#Total pagos Liver registrados
pagos_totales_liv = pd.pivot_table(pagos_liv2,index=['Cliente'],values=['PagoArancel'],aggfunc=np.sum,dropna=False)
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
#pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv.columns = ['Cliente','Facturacion']
pagos_totales_liv.fillna(0,inplace=True)
pagos_totales_liv['Facturacion'] = pagos_totales_liv['Facturacion'].round(2)
pagos_totales_liv['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv['FechaFact'] = pd.to_datetime(pagos_totales_liv['FechaFact'],dayfirst=True)
pagos_totales_liv

,Cliente,Facturacion,FechaFact
0,Liverpool,1014128.05,2020-12-27


# Bradesco

In [80]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 1204704, .07
c04pv2, arancel_4pv2 = 1807056, .09
c04pv3, arancel_4pv3 = 2409407, .12
c05pv1, arancel_5pv1 = 662269, .09
c05pv2, arancel_5pv2 = 993403, .10
c05pv3, arancel_5pv3 = 1324538, .14
c06pv1, arancel_6pv1 = 807172, .10
c06pv2, arancel_6pv2 = 1210759, .11
c06pv3, arancel_6pv3 = 1614345, .16
a1, arancel_a1 = 260190, .27
a2, arancel_a2 = 390285, .30
a3, arancel_a3 = 520381, .33
b1, arancel_b1 = 215083, .25
b2, arancel_b2 = 322624, .28
b3, arancel_b3 = 430165, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 626634, .19
c22, arancel_c22 = 939951, .22
c23, arancel_c23 = 1253267, .25
c31, arancel_c31 = 246425, .21
c32, arancel_c32 = 369637, .24
c33, arancel_c33 = 492849, .27
c41, arancel_c41 = 123529, .23
c42, arancel_c42 = 185294, .26
c43, arancel_c43 = 247059, .29

In [77]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [78]:
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [81]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.03)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.03)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.03)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.03)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.03)))
    
    else:
        return 0

pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)

In [82]:
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]
pagos_brad2['Cliente'] = 'Bradesco'

In [86]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad2,index=['Cliente'],values=['Facturacion'],aggfunc=[np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2.columns = ['Cliente','Facturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_brad2

,Cliente,Facturacion,FechaFact
0,Bradesco,502890.3666,2020-12-27


# Credifiel

In [87]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_pagos_cred = pd.merge(pagos_cred2,asignacion_cred,how='left',left_on=['ID'],right_on=['CREDITO'])
consolidado_pagos_cred.drop('CREDITO',1,inplace=True)
consolidado_pagos_cred['ARANCEL'] = np.where(consolidado_pagos_cred['ARANCEL'].isna(),np.mean(consolidado_pagos_cred['ARANCEL']),consolidado_pagos_cred['ARANCEL'])
consolidado_pagos_cred['FACTURACION'] = consolidado_pagos_cred['IMPORTE']*consolidado_pagos_cred['ARANCEL']
consolidado_pagos_cred['CLIENTE'] = 'Credifiel'
consolidado_pagos_cred.head(1)

,ID,FECHA,IMPORTE,ARANCEL,FACTURACION,CLIENTE
0,31042,2020-12-18,1500.0,0.4,600.0,Credifiel


In [90]:
pagos_totales_cred = pd.pivot_table(consolidado_pagos_cred,index=['CLIENTE'],values=['FACTURACION'],aggfunc=[np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
#pagos_totales_cred = pagos_totales_cred.iloc[:,[0,1,4,3]]
pagos_totales_cred.columns = ['Cliente','Facturacion']
pagos_totales_cred.fillna(0,inplace=True)
#pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
pagos_totales_cred['FechaFact'] = max(consolidado_pagos_cred['FECHA'])
pagos_totales_cred

,Cliente,Facturacion,FechaFact
0,Credifiel,46668.940422,2020-12-18


# Banco Azteca

In [91]:
pagos_baz = pd.concat([pagos_baz_26,pagos_baz_recovery],ignore_index=True)
pagos_baz = pagos_baz.loc[pagos_baz['CARGO AUTOMATICO']=='NO']
pagos_baz['Recuperado'] = pagos_baz['RECUPERACION DE CAPITAL']+pagos_baz['RECUPERACION DE MORATORIOS']
pagos_baz['Cliente'] = 'Banco Azteca'

In [92]:
pagos_totales_baz = pd.pivot_table(fact_baz,index=['Cliente'],values=['Total Recuperado','Total Comisión'],aggfunc=[np.sum],dropna=False)
pagos_totales_baz = pd.DataFrame(pagos_totales_baz.to_records())
pagos_totales_baz.columns = ['Cliente','Recuperado','SumaPagos']
porcentaje_prom_arancel_baz = pagos_totales_baz['Recuperado']/pagos_totales_baz['SumaPagos']
porcentaje_prom_arancel_baz

0    0.146855
dtype: float64

In [98]:
pagos_baz['Facturacion'] = [x*porcentaje_prom_arancel_baz for x in pagos_baz['Recuperado']]
pagos_fact_baz = pd.pivot_table(pagos_baz,index=['Cliente'],values=['Facturacion'],aggfunc=[np.sum])
pagos_fact_baz = pd.DataFrame(pagos_fact_baz.to_records())
pagos_fact_baz.columns = ['Cliente','Facturacion']
pagos_fact_baz['FechaFact'] = max(pagos_baz['DIA'])
pagos_fact_baz

,Cliente,Facturacion,FechaFact
0,Banco Azteca,387066.770176,2020-12-27
